In [1]:
import pandas
import datetime
import numpy

In [2]:
datFileName = "../data/processedSaps.txt"
sapsDataDF = pandas.read_csv(datFileName, sep=' ')
# # add dst_bins
dstBins = [ -150, -75, -50, -25, -10, 10 ]
sapsDataDF = pandas.concat( [ sapsDataDF, \
                    pandas.cut( sapsDataDF["dst_index"], \
                               bins=dstBins ) ], axis=1 )
sapsDataDF.columns = [ "dateStr", "sapsLat", "sapsMLT", \
                      "sapsVel", "radId", "poesLat", "poesMLT", \
                      "dst_date", "dst_index", "time", "dst_bin" ]
sapsDataDF = sapsDataDF.drop(["poesLat", "poesMLT"], 1)
sapsDataDF["sapsMLTRounded"] = sapsDataDF["sapsMLT"].map(lambda x: round(x) )
sapsDataDF.head()

,dateStr,sapsLat,sapsMLT,sapsVel,radId,dst_date,dst_index,time,dst_bin,sapsMLTRounded
0,20110107,56.5,17.7543,308.2077,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0
1,20110107,55.5,18.0147,224.1588,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0
2,20110107,56.5,17.8749,307.4328,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0
3,20110107,55.5,18.1324,222.4787,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0
4,20110107,56.5,17.9955,305.4201,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0


In [3]:
sapsNumRadsSer = sapsDataDF.groupby( ["dateStr"] ).agg( {"radId": pandas.Series.nunique} )

In [4]:
# get times, dates when the number of data points observed
# is greater than 50 points at a given time.
sapsDateTimeCount = sapsDataDF.groupby(["dateStr", "time"]).count()
sapsDateTimeCount = sapsDateTimeCount[ sapsDateTimeCount["sapsLat"] > 50 ].reset_index()
sapsDateTimeCount = sapsDateTimeCount[['dateStr','time', 'sapsLat']]
sapsDateTimeCount.columns = [ "dateStr", "time", "nPoints" ]
# Now get data points when atleast 4 radars were making observations
sapsUniqRadsCnt = sapsDataDF.groupby(["dateStr","time"]).agg( {"radId": pandas.Series.nunique} )
sapsUniqRadsCnt = sapsUniqRadsCnt[ sapsUniqRadsCnt >= 4 ].dropna().reset_index()
sapsUniqRadsCnt.columns = [ "dateStr", "time", "nRads" ]
# Merge both to get good dates where we can get velocity measurements
selVelDatesDF = pandas.merge( sapsUniqRadsCnt, \
                             sapsDateTimeCount, \
                             on=[ "dateStr", "time" ], how="inner" )
selVelDatesDF.head()

,dateStr,time,nRads,nPoints
0,20110205,230,4.0,98
1,20110205,300,4.0,88
2,20110205,330,5.0,93
3,20110302,100,4.0,75
4,20110302,200,4.0,81


In [5]:
def convert_to_datetime(row, datecolName="dateStr", timeColName="time"):
    currDateStr = str( int( row[datecolName] ) )
#     return currDateStr
    if row[timeColName] < 10:
        currTimeStr = "000" + str( int( row[timeColName] ) )
    elif row[timeColName] < 100:
        currTimeStr = "00" + str( int( row[timeColName] ) )
    elif row[timeColName] < 1000:
        currTimeStr = "0" + str( int( row[timeColName] ) )
    else:
        currTimeStr = str( int( row[timeColName] ) )
    return datetime.datetime.strptime( currDateStr\
                    + ":" + currTimeStr, "%Y%m%d:%H%M" )

In [6]:
# Merge the selected date time DF with sapsDataDF
# to get the radar ids as well
selVelDatesDF = pandas.merge( selVelDatesDF, sapsDataDF, on=["dateStr","time"] )
selVelDatesDF = selVelDatesDF[ [ 'dateStr','time', 'radId' ] ]
selVelDatesDF = selVelDatesDF.drop_duplicates()
selVelDatesDF["dtObj"] = selVelDatesDF.apply( convert_to_datetime, axis=1 )
# selVelDatesDF.to_csv("../data/XuelingSaps.txt", sep=' ', index=False)
selVelDatesGrps = selVelDatesDF.groupby( ["dateStr", "radId"] )
selVelDatesDF.head()

,dateStr,time,radId,dtObj
0,20110205,230,206.0,2011-02-05 02:30:00
20,20110205,230,207.0,2011-02-05 02:30:00
29,20110205,230,205.0,2011-02-05 02:30:00
36,20110205,230,33.0,2011-02-05 02:30:00
98,20110205,300,206.0,2011-02-05 03:00:00


In [7]:
velDateTimeRadDF = selVelDatesGrps.aggregate(lambda x: tuple(x)).reset_index()
velDateTimeRadDF["timeRange"] = velDateTimeRadDF["time"].map(\
                                    lambda x: [min(x), max(x)] )
velDateTimeRadDF["minTime"] = velDateTimeRadDF["time"].map(\
                                    lambda x: min(x) )
velDateTimeRadDF["maxTime"] = velDateTimeRadDF["time"].map(\
                                    lambda x: max(x) )
# Now IDL is messing things up! when reading dates.
# So have year, month and day columns to read the dates!!!
# velDateTimeRadDF["dtYear"] = [ x/10000 for x in velDateTimeRadDF["dateStr"] ]
# velDateTimeRadDF["dtMon"] = [ (x-(x/10000)*10000)/100 \
#                              for x in velDateTimeRadDF["dateStr"] ]
# velDateTimeRadDF["dtDay"] = [ ( (x-(x/10000)*10000) - ((x-(x/10000)*10000)/100)*100 ) \
#                              for x in velDateTimeRadDF["dateStr"] ]
# save the data in a csv file to process using IDL for getting velocities
velDateTimeRadDF[ ['dateStr', 'radId', 'minTime', 'maxTime'] \
                ].to_csv("../data/sapsVelDates.txt", sep=' ', index=False)
velDateTimeRadDF.head()

,dateStr,radId,time,dtObj,timeRange,minTime,maxTime
0,20110205,33.0,"(230, 300, 330)","(2011-02-05 02:30:00, 2011-02-05 03:00:00, 201...","[230, 330]",230,330
1,20110205,204.0,"(330,)","(2011-02-05 03:30:00,)","[330, 330]",330,330
2,20110205,205.0,"(230, 300, 330)","(2011-02-05 02:30:00, 2011-02-05 03:00:00, 201...","[230, 330]",230,330
3,20110205,206.0,"(230, 300, 330)","(2011-02-05 02:30:00, 2011-02-05 03:00:00, 201...","[230, 330]",230,330
4,20110205,207.0,"(230, 300, 330)","(2011-02-05 02:30:00, 2011-02-05 03:00:00, 201...","[230, 330]",230,330


In [8]:
# Get the difference between consecutive elements in the time tuple
# We can determine the time ranges for each radar this way...
velDateTimeRadDF["timeDiffs"] = velDateTimeRadDF["dtObj"].map(\
                                    lambda x: numpy.ediff1d(x) )
# velDateTimeRadDF['timeDiffs'] = [max(x) if len(x) > 0 else datetime.timedelta(minutes=0) for x in velDateTimeRadDF['timeDiffs']]
# velDateTimeRadDF.head()
# Pivot the table so that date is row,
# radId is col and time is the value!!!
pivDF = velDateTimeRadDF.pivot(index='dateStr', columns='radId', values='time')
pivDF.head()

radId,32.0,33.0,204.0,205.0,206.0,207.0,208.0,209.0
dateStr,,,,,,,,
20110205,None,"(230, 300, 330)","(330,)","(230, 300, 330)","(230, 300, 330)","(230, 300, 330)",None,None
20110302,"(100, 200, 230)","(100, 200, 230)",None,"(200, 230)","(100, 230)","(100, 200, 230)",None,None
20110310,"(600,)","(600,)",None,"(600,)","(600,)","(600,)",None,None
20110312,"(30,)","(30,)",None,"(30,)",None,"(30,)",None,None
20110403,"(400, 430, 500, 530, 600)","(400, 430, 500, 530, 600)",None,None,"(400, 430, 500, 530, 600)","(400, 430, 500, 530, 600)",None,None


In [9]:
# This is a SAPS list for Jo
def to_dt_minTime(row, datecolName="dateStr", timeColName="minTime"):
    currDateStr = str( int( row[datecolName] ) )
#     return currDateStr
    if row[timeColName] < 10:
        currTimeStr = "000" + str( int( row[timeColName] ) )
    elif row[timeColName] < 100:
        currTimeStr = "00" + str( int( row[timeColName] ) )
    elif row[timeColName] < 1000:
        currTimeStr = "0" + str( int( row[timeColName] ) )
    else:
        currTimeStr = str( int( row[timeColName] ) )
    return datetime.datetime.strptime( currDateStr\
                    + ":" + currTimeStr, "%Y%m%d:%H%M" )

def to_dt_maxTime(row, datecolName="dateStr", timeColName="maxTime"):
    currDateStr = str( int( row[datecolName] ) )
#     return currDateStr
    if row[timeColName] < 10:
        currTimeStr = "000" + str( int( row[timeColName] ) )
    elif row[timeColName] < 100:
        currTimeStr = "00" + str( int( row[timeColName] ) )
    elif row[timeColName] < 1000:
        currTimeStr = "0" + str( int( row[timeColName] ) )
    else:
        currTimeStr = str( int( row[timeColName] ) )
    return datetime.datetime.strptime( currDateStr\
                    + ":" + currTimeStr, "%Y%m%d:%H%M" )

In [15]:
dateMin = velDateTimeRadDF.groupby( ["dateStr"] )["minTime"].min()
dateMax = velDateTimeRadDF.groupby( ["dateStr"] )["maxTime"].max()
sapsDatesDF = pandas.concat( [dateMin, dateMax], axis=1 ).reset_index()
# If the start time and end time are the same then subtract 15 min 
# from min time and add 15 min to max time. I know this ain't the 
# most efficient way to go about it! But its a quick and dirty sol.
sapsDatesDF["mindtObj"] = sapsDatesDF.apply( to_dt_minTime, axis=1 )
sapsDatesDF["maxdtObj"] = sapsDatesDF.apply( to_dt_maxTime, axis=1 )
sapsDatesDF["mindtObjNew"] = [x if x != y else x-datetime.timedelta(minutes=15) \
                           for x,y in zip(sapsDatesDF['mindtObj'],sapsDatesDF['maxdtObj'])]
sapsDatesDF["maxdtObj"] = [x if x != y else x+datetime.timedelta(minutes=15) \
                           for x,y in zip(sapsDatesDF['maxdtObj'],sapsDatesDF['mindtObj'])]
sapsDatesDF["minTime"] = sapsDatesDF["mindtObjNew"].map(\
                                    lambda x: x.strftime("%H%M") )
sapsDatesDF["maxTime"] = sapsDatesDF["maxdtObj"].map(\
                                    lambda x: x.strftime("%H%M") )
sapsDatesDF = sapsDatesDF[ ["dateStr", "minTime", "maxTime"] ]
sapsDatesDF.columns = [ "date", "start_time", "end_time" ]
sapsDatesDF.to_csv("../data/joSapsDates.txt", sep=' ', index=False)
sapsDatesDF.head()

,date,start_time,end_time
0,20110205,0230,0330
1,20110302,0100,0230
2,20110310,0545,0615
3,20110312,0015,0045
4,20110403,0400,0600
